# LSTM as Autoencoder for Compute FCI


Note: Kernel en pc Ivan = env_nvn_rag(python 3.10.13) ya que tiene pytorch cuda
      Kernel en pc Sandra =

In [107]:
import clean
import numpy as np
import torch
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import random

get the determinants and format them into a numpy array

In [108]:
name='psi_det'
so_vectors=clean.clean(name)
so_vectors=np.array(so_vectors,dtype=np.float32)
display(so_vectors[:3])
so_vectors=torch.tensor(so_vectors)

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [109]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

class Encoder(nn.Module):

  def __init__(self, seq_len, n_features, embedding_dim=64):
    super(Encoder, self).__init__()
    self.seq_len, self.n_features = seq_len, n_features
    self.embedding_dim, self.hidden_dim = embedding_dim, 2 * embedding_dim

    self.rnn1 = nn.LSTM(
      input_size=n_features,
      hidden_size=self.hidden_dim,
      num_layers=1,
      batch_first=True
    )
    
    self.rnn2 = nn.LSTM(
      input_size=self.hidden_dim,
      hidden_size=embedding_dim,
      num_layers=1,
      batch_first=True
    )

  def forward(self, x):
    x, (_, _) = self.rnn1(x)
    x, (hidden_n, _) = self.rnn2(x)
    return x
  
class Decoder(nn.Module):
  def __init__(self, seq_len,input_dim=64, n_features=8):
    super(Decoder, self).__init__()

    self.seq_len, self.input_dim = seq_len, input_dim
    self.hidden_dim, self.n_features = 2 * input_dim, n_features

    self.rnn1 = nn.LSTM(
      input_size=input_dim,
      hidden_size=input_dim,
      num_layers=1,
      batch_first=True
    )

    self.rnn2 = nn.LSTM(
      input_size=input_dim,
      hidden_size=self.hidden_dim,
      num_layers=1,
      batch_first=True
    )

    self.output_layer = nn.Linear(self.hidden_dim, n_features)
  def forward(self, x):
    x, (hidden_n, cell_n) = self.rnn1(x)
    x, (hidden_n, cell_n) = self.rnn2(x)
    x = x.reshape((-1,self.seq_len, self.hidden_dim))  
    x = F.normalize(torch.abs(self.output_layer(x)), p=1, dim=1)

    return x 
  

class RecurrentAutoencoder(nn.Module):
    def __init__(self, seq_len,ne, n_features,embedding_dim=64):
        super(RecurrentAutoencoder, self).__init__()
        self.encoder = Encoder(seq_len, n_features, embedding_dim).to(device)
        self.decoder = Decoder(seq_len, embedding_dim, n_features).to(device)
        self.ne=ne

      
    #my own function to apli to the final output of decoder
    def electron_constriction(self, x): #use [-1,:,-1] to get the last value of the last sequence
        x2=x[-1,:  ,-1]
        x2=torch.abs(x2)
        zero_vector=torch.zeros(x2.shape,dtype=torch.float32)

        for i in range (self.ne//2):
          #----------------------------constriction for even  positions (electrons in alpha orbitals)---------------------
          random_value=torch.rand(1).to(device)
          #cumsum of odd values
          odd_values=torch.cumsum(x2[::2],0)
          random_values=random_value*odd_values[-1] #random values in range of cumsum


          #get the index of the closest value to the random value using argmax
          mask=odd_values>=random_values
          Index_where_mask_is_true = (torch.nonzero(mask, as_tuple=True)[0])*2  # is like argmax but for boolean in pytorch
          jump=Index_where_mask_is_true[0]

          x2[jump]=0  #set to 0 the prbability in x2
          zero_vector[jump]=1 #set to 1 the position of the jump in the vector x


          #----------------------------constriction for odd  positions (electrons in beta orbitals)---------------------
          random_value=torch.rand(1).to(device)
          even_values=torch.cumsum(x2[1::2],0)
          random_values=random_value*even_values[-1] #random values in range of cumsum

          #get the index of the closest value to the random value using argmax
          mask=even_values>=random_values
          Index_where_mask_is_true = (torch.nonzero(mask, as_tuple=True)[0])*2+1  # is like argmax but for boolean in pytorch
          jump=Index_where_mask_is_true[0]

          x2[jump]=0  #set to 0 the prbability in x2
          zero_vector[jump]=1 #set to 1 the position of the jump in the vector x
          
        if torch.count_nonzero(zero_vector)!=self.ne:
          print('error numero de electrones en vector es:',torch.count_nonzero(zero_vector))

        x[-1,:,-1]=zero_vector  #set the vector with the constriction of electrons

        #x to device cuda
        return x.to(device)
  

    def forward(self, x):    
        x = self.encoder(x)
        x = self.decoder(x)
        x=self.electron_constriction(x)
        return x

In [110]:
class TimeSeriesDataset(Dataset):
    def __init__(self, data, seq_len):
        self.data = data
        self.seq_len = seq_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [111]:
def train_model(model, train_loader,  n_epochs,lr):
  optimizer = torch.optim.Adam(model.parameters(), lr)
  criterion = nn.L1Loss(reduction='mean').to(device) #analizies MAE between prediction and target
  history = dict(train=[], val=[])

  for epoch in range(1, n_epochs + 1):
    model = model.train()
    print(f'\rÉpoca: {epoch}', end='', flush=True)
    train_losses = []
    for seq_true in train_loader:
      optimizer.zero_grad()
      seq_true = seq_true.to(device)
      seq_pred = model(seq_true)
      loss = criterion(seq_pred, seq_true)
      loss.backward()
      optimizer.step()
      train_losses.append(loss.item())
    model = model.eval()
   
  return model.eval(), history

In [112]:
seq_len=26
features=1
num_samples = len(so_vectors) 
tensor_data = so_vectors[:num_samples * seq_len]
tensor_data = tensor_data.reshape((num_samples, seq_len, features))
indices = torch.randperm(num_samples)
tensor_data = tensor_data[indices]
train_dataset = TimeSeriesDataset(tensor_data, seq_len)
batch_size=64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [113]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = RecurrentAutoencoder(26,10, 1,64)
model = model.to(device)
model, history = train_model(
  model, 
  train_loader,  
  n_epochs=10,
  lr=0.001
)

cuda
Época: 10

In [114]:
torch.save(model.state_dict(), 'recurrent_autoencoder.pth')

In [115]:
# Load the entire state_dict
state_dict = torch.load('recurrent_autoencoder.pth')
device = torch.device("cpu")
# Initialize the decoder
decoder = Decoder(seq_len=26, input_dim=64, n_features=1).to(device)

# Load only the decoder weights
decoder_state_dict = {k.replace('decoder.', ''): v for k, v in state_dict.items() if k.startswith('decoder.')}
decoder.load_state_dict(decoder_state_dict)

<All keys matched successfully>

# Generation of new determinants

In [186]:
def generation(decoder,Ne,dets_train):
    random_data=torch.randn(1, 26, 64)
    decoded_output = decoder(random_data)
    visible_probs = decoded_output.detach().numpy()
    #formated to 4 decimals
    #print('probabilidades visibles',np.around(visible_probs[0],5))
    #print the 5 orbitals with the highest probabilities
    a=np.argsort(visible_probs,axis=1)
    #print(visible_probs)
    print('los 5 orbitales con mayor probabilidad son',a[0][-5:])

    #random num to singlet or doublet
    exitations=0
    random_exitation= np.random.randint(1)
    if random_exitation>0.5:
        exitations=1
    else:
        exitations=2

    print('numero de excitaciones',exitations)

    random_number_det=np.random.randint(len(dets_train))
    det=dets_train[random_number_det]   #choose a random determinant from the training set
    print('Tensor 1',det)

    #position of the occupied and unoccupied orbitals
    occupied_orbitals=np.where(det==1)[0]
    unoccupied_orbitals=np.where(det==0)[0]

    #compute the conditional probabilities for a change of 1 electron in a occupied orbital to a unoccupied orbital
    jumps=np.zeros((len(occupied_orbitals),len(unoccupied_orbitals)))
    for j in range(len(occupied_orbitals)):
        for k in range(len(unoccupied_orbitals)):
            #probabilities of the jumps, (1 - prob of the occupied orbital being ocuped)*(prob of the unoccupied orbital being ocuped)
            #this is the same as: probability of the occupied orbital being unocuped * probability of the unoccupied orbital being ocuped
            jumps[j][k]=(1-visible_probs[0][occupied_orbitals[j]])*visible_probs[0][unoccupied_orbitals[k]]


    #sample the jumps
    new_det = det.clone() #copy the determinant to change it
    saved_jumps=[]
    for i in range(exitations):
        c=0 #acumulative sum of probabilities
        #p=np.random.random()*np.sum(jumps) #random number between 0 and the sum of all the probabilities in the jumps matrix
        p=random.random()*np.sum(jumps) #creo que funciona mejor en el paralelismo
        
        found_change=False

        for j in range(len(occupied_orbitals)):
            for k in range(len(unoccupied_orbitals)):
                c+=jumps[j][k]
                
                #if the random number is less or equal to the acumulative sum of probabilities, then we change the determinant
                # the restriction jumps[j][k]!=0 is to avoid to choose the same occupied and unoccupied orbital, that previously was changed to 0
                #for example (j=7 y k= 30 suma acumulada 112.07108543752757) y  (j=7 y k =31 suma acumulada 112.07108543752757), the probabilities 
                #are the same, it means that the occupied orbital 7 was changed to 0, and the unoccupied orbital 30 was changed to 0, so we cannot choose
                if p<=c and jumps[j][k]!=0: 
                    #print('cambio de orbital',occupied_orbitals[j],'por',unoccupied_orbitals[k])
                
                    #check if the occupied and unoccupied orbitals have the same spin (alpha electron in alpha orbital or beta electron in beta orbital)
                    #if (j%2==0 and k%2==0) or (j%2!=0 and k%2!=0):
                    if (occupied_orbitals[j]%2==0 and unoccupied_orbitals[k]%2==0) or (occupied_orbitals[j]%2!=0 and unoccupied_orbitals[k]%2!=0):
                        #change the determinant
                        new_det[occupied_orbitals[j]]=0
                        new_det[unoccupied_orbitals[k]]=1

                        #change the probabilities for this orbitals to 0, to avoid to choose them again
                        #jumps[j][:]=0
                        jumps[j]=0
                        jumps[:,k]=0
                        #print('las probabilidades de los jumps intermedias son',jumps)

                        found_change=True

                        #save the jumps
                        saved_jumps.append([occupied_orbitals[j],unoccupied_orbitals[k]])
                        
                        break

            if found_change:
                break
        if not found_change:
            print('no se encontro cambio, es posible que haya un error en el algoritmo******************************')
            continue

    return new_det
    

In [188]:
new_det_generated=generation(decoder,10,so_vectors)
print('Tensor 2',new_det_generated)

los 5 orbitales con mayor probabilidad son [[24]
 [ 8]
 [25]
 [10]
 [ 9]]
numero de excitaciones 2
Tensor 1 tensor([1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0.])
Tensor 2 tensor([1., 1., 1., 1., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0.])
